# RSpace API practical

You can use the RSpace API through python to create, list, upload, and manage many aspects of your RSpace objects. However, some features can only be done through the RSpace user interface online.

Use of the API can be complicated if all you want to do is perform simple tasks. However, if you want to e.g. replicate the same task many times, or if you want to integrate RSpace into your analysis workflow by automatically upload any results/plots/documents etc, the API can help and improve your efficiency and robustness.

The Python API tools can be found on GitHub at [https://github.com/rspace-os/rspace-client-python](https://github.com/rspace-os/rspace-client-python). You should refer to the official github RSpace repository in the future, as the present notebook may run out of date.

You can also read more about the RSpace API at [https://rspace.ntnu.no/public/apiDocs](https://rspace.ntnu.no/public/apiDocs).

## Requirements

Use of the API through python depends on the `rspace_client==2.5.0` package found on both PyPi and conda. In addition, a few other requirements are useful. For __TEM__ analysis, it is advised that a repository for RSpace API use includes at least these packages:

- rspace_client==2.5.0
- pyxem==0.19.1
- hyperspy
- hyperspy-gui-ipywidgets
- hyperspy-gui-traitsui
- tabulate
- jupyter
- jupyterlab
- matplotlib
- numpy<2.0.0
- scipy
- markdown
- json2html
- pymdown-extensions

## Preparations

In order to use the API through python, you will need to do some preparations first. Let us start by setting up an environment.

### Setting up environment

Create a new environment for _pure_ RSpace use through conda:

```bash
conda create -n RSpace -c conda-forge rspace_client=2.5.0 tabulate jupyter jupyterlab matplotlib numpy<2.0.0 scipy markdown json2html pymdown-extensions
```

#### Pip

The RSpace client can also be installed through pip:

```bash
conda create -n RSpace pip pyxem=0.19.1 hyperspy hyperspy-gui-traitsui hyperspy-gui-ipywidgets tabulate jupyter jupyterlab matplotlib numpy<2.0.0 scipy markdown json2html pymdown-extensions
conda activate RSpace
pip install rspace_client==2.5.0
```

### API Key
Next, you need to get your API key from RSpace. Your API key is _equivalent_ to a password, and should be treated as a secret. A few key points to remember is:

- __NEVER__ share your API key
- __NEVER__ include your API key directly in scripts or notebooks
- Be __CAREFUL__ if you enter your API key as input to scripts through the command-console. It will be stored in readable clear text in the console log!

We will get back to how to use your API key in a _more_ secure way later on. 

#### Getting your API Key

To get your API key:

1. Log in to RSPace at [https://rspace.ntnu.no](https://rspace.ntnu.no)
2. Go to "My RSpace"
3. Go to "My Profile"
4. Scroll to the bottom and click "Generate API key"
5. Save the API key in a secure way. It will not be displayed again! Anyone with access to the API key will have full control of your RSpace.

If your API key for some reason gets compromised, or if you lose it, you can always repeat the steps above to regenerate the key. This will render any previous API key invalid. You can also revoke the key if you want to disable API access altogether.

#### Adding your key to the system environmental variables

To use your API key in scripts or on notebooks without adding it as clear text to files or in the command line, you can instead add the API key to the system environmental variables. This allows you to get the API key through python with commands such as 

```python
import os
os.getenv("RSPACE_API_KEY")
```

How to add your API key to the system environmental variables varies from system to system.

##### Windows

On Windows, you can add your API key to the system environmental variables by (see example images after the list):

1. Search for "environment" in the start menu (you should get a hit for "Edit the system environment variables").
2. Under "User variables for ..." click "New".
3. Enter a suitable variable name (e.g. "RSPACE_API_KEY") and enter the API key you got from RSpace as the value.

<img src="../images/windows_edit_env_var_1.PNG" alt="Edit system environmental variables Windows 1" width="200"/>  
  
<img src="../images/windows_edit_env_var_2.PNG" alt="Edit system environmental variables Windows 2" width="200"/>  
  
<img src="../images/windows_edit_env_var_3.PNG" alt="Edit system environmental variables Windows 3" width="200"/>  

##### Linux

In linux you can add your RSpace API key to the system environment variables by editing your bashrc file (note that this involves storing the key in an unsecured text file and poses a potential security risk). 

Open bashrc in a text editor and add `export RSPACE_API_KEY=key` anywhere in the file. You can e.g. open the bashrc file with nano by running this command:
```bash
nano ~/.bashrc
```

## Using the API

You should now be able to use this notebook. Use of the API is relatively straight forward, and you can usually find good examples and demos at [https://github.com/rspace-os/rspace-client-python/tree/master/examples](https://github.com/rspace-os/rspace-client-python/tree/master/examples). 

Most of the API commands returns a dictionary with the response from the RSpace server.

In [188]:
from matplotlib import pyplot as plt
%matplotlib notebook

import os
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import markdown
import json2html
import pandas as pd
import seaborn as sns

import hyperspy.api as hs
import pyxem as pxm

from rspace_client.eln.eln import ELNClient

def get_url_link(response):
    """
    Extracts a URL from an API response.
    
    Parameters
    ----------
    response: dict
        The API response.

    Returns
    -------
    url : str
        URL to the object on RSpace.

    """
    base_url = response['_links'][0]['link'].split('/api', 1)[0]
    return f"{base_url}/globalId/{response['globalId']}/"

In [2]:
rspace_url = r"https://rspace.ntnu.no/"
api_key = os.getenv("RSPACE_API_KEY")
client = ELNClient(rspace_url, api_key)

## List documents
Get information about the documents on RSpace. The returned value is a dictionary with lots of information as well as a _paginated_ list of document entires. The _paginated_ list is restricted to 20 entires by default (you can change this by supplying an argument to `client.get_documents()`). To get the other documents, you must make additional calls to `client.get_documents(page_number=i)`.

In [3]:
response = client.get_documents()
response

{'totalHits': 43,
 'pageNumber': 0,
 'documents': [{'id': 3556,
   'globalId': 'SD3556',
   'name': 'test',
   'created': '2025-01-17T06:54:12.546Z',
   'lastModified': '2025-01-17T07:06:48.958Z',
   'parentFolderId': None,
   'signed': False,
   'tags': 'API',
   'tagMetaData': None,
   'form': {'id': 65536,
    'globalId': 'FM65536',
    'stableId': '1724164328846null',
    'version': 0,
    'name': 'Basic Document',
    'tags': None,
    'formState': 'PUBLISHED',
    'accessControl': {'ownerPermissionType': 'WRITE',
     'groupPermissionType': 'NONE',
     'worldPermissionType': 'READ'},
    'iconId': -1,
    '_links': []},
   'owner': {'id': 196610,
    'username': 'emilc@ntnu.no',
    'email': 'emil.christiansen@ntnu.no',
    'firstName': 'Emil Frang',
    'lastName': 'Christiansen',
    'homeFolderId': 615,
    'workbenchId': None,
    'hasPiRole': True,
    'hasSysAdminRole': False,
    '_links': []},
   '_links': [{'link': 'https://rspace.ntnu.no/api/v1/documents/3556',
     'r

In [8]:
print(f"Found {response['totalHits']} documents on RSpace. The response contains information about {len(response['documents'])} documents. To see information about more documents, run `client.get_documents(page_number=i)` with `i>0`.")
_ = [print(document['id'], document['name']) for document in response['documents']]

Found 43 documents on RSpace. The response contains information about 20 documents. To see information about more documents, run `client.get_documents(page_number=i)` with `i>0`.
3556 test
3554 test
3553 Untitled document
3552 Untitled document
3474 example_experiment
3466 example_experiment
3338 Untitled document
2921 PicoScope measurements
3124 File delete from gallery
2648 example_experiment
2868 Untitled document
674 AA6060 T6 AC heat treatment
2647 Untitled document
2632 Untitled document
2594 example_experiment
2577 example_experiment
2364 Untitled document
2378 NORTEM ontologies
2306 Python API Example Basic Document
2304 Python API Example Basic Document


To get all the documents on RSpace, we need to make consecutive calls to `client.get_documents()` with increasing page_numbers. One way to do this without knowing how many pages we expect there are, is to run an infinite loop that we break once the response stops returning any documents

In [10]:
documents = []
i = 0
while True:
    response = client.get_documents(page_number=i)
    if len(response["documents"]) == 0:
        break
    documents += response["documents"]
    i += 1
documents

[{'id': 3556,
  'globalId': 'SD3556',
  'name': 'test',
  'created': '2025-01-17T06:54:12.546Z',
  'lastModified': '2025-01-17T07:06:48.958Z',
  'parentFolderId': None,
  'signed': False,
  'tags': 'API',
  'tagMetaData': None,
  'form': {'id': 65536,
   'globalId': 'FM65536',
   'stableId': '1724164328846null',
   'version': 0,
   'name': 'Basic Document',
   'tags': None,
   'formState': 'PUBLISHED',
   'accessControl': {'ownerPermissionType': 'WRITE',
    'groupPermissionType': 'NONE',
    'worldPermissionType': 'READ'},
   'iconId': -1,
   '_links': []},
  'owner': {'id': 196610,
   'username': 'emilc@ntnu.no',
   'email': 'emil.christiansen@ntnu.no',
   'firstName': 'Emil Frang',
   'lastName': 'Christiansen',
   'homeFolderId': 615,
   'workbenchId': None,
   'hasPiRole': True,
   'hasSysAdminRole': False,
   '_links': []},
  '_links': [{'link': 'https://rspace.ntnu.no/api/v1/documents/3556',
    'rel': 'self'}]},
 {'id': 3554,
  'globalId': 'SD3554',
  'name': 'test',
  'created

We can also create a table with the document IDs and the titles to format it a little nicer than before.

In [12]:
table = tabulate([[document['id'], document['name']] for document in documents], ['ID', 'Title'])

print(table)

  ID  Title
----  ---------------------------------------------
3556  test
3554  test
3553  Untitled document
3552  Untitled document
3474  example_experiment
3466  example_experiment
3338  Untitled document
2921  PicoScope measurements
3124  File delete from gallery
2648  example_experiment
2868  Untitled document
 674  AA6060 T6 AC heat treatment
2647  Untitled document
2632  Untitled document
2594  example_experiment
2577  example_experiment
2364  Untitled document
2378  NORTEM ontologies
2306  Python API Example Basic Document
2304  Python API Example Basic Document
2231  SJHP3 ELISA
2229  bam mutant rtPCR
2226  Sensitivity of Cancer cell line XYZ to Drug A
2225  Extract Protocol for ChIP
2069  SJHP3 ELISA
2067  bam mutant rtPCR
2064  Sensitivity of Cancer cell line XYZ to Drug A
2063  Extract Protocol for ChIP
1846  SPED data aquisition
1845  SPED data aquisition
 779  Goniometer characterization
 671  AA6060 T6 WQ heat treatment
 652  YOUR_AUTOGENERATED_TAGS.ontology
 642  SJHP3 

## Create a document with the API

You can also create documents on RSpace through the API with the `client.create_document()` call. All documents on RSpace are actually _forms_. A _form_ is a group or collection of _fields_ that can hold information. This means that when creating a document you will need to specify which _field_ you want to put various pieces of information. Let us first try to create a simple basic document with some text.

In [14]:
text = "<h1>RSpace API example text</h1>\nThis is a very useful document <h2>It can have various header levels</h2>\nIf you write some HTML syntax in here, it will be nicely formatted once it goes on RSpace."

response = client.create_document(name="Test", parent_folder_id=None, tags=["API", "Test"], fields=[{"content": text}])

print(f"Created document with ID {response['id']}")

Created document with ID 3559


If you go to RSpace you will now be able to find a document with the ID printed above. You can now edit the document on RSpace directly, or continue working on it through the API

## Do some analysis and update the document

In [20]:
output_file = Path("test.png")
x = np.arange(0, 4*np.pi, 0.1)
y = np.sin(x)
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Sine wave')
fig.savefig("test.png")

with output_file.open('rb') as f:
    upload_response = client.upload_file(f, caption="API Test")
print(f"Uploaded image with ID {upload_response['id']}")

<IPython.core.display.Javascript object>

Uploaded image with ID 3562


In [55]:
content = f"""
<h2>Example of python analysis</h2>
<p>Plot of a sine wave between {x[0]} and {x[-1]}: <fileId={upload_response['id']}></p>
"""

updated_doc = client.append_content(response['id'], content)

print(f"Updated document ({updated_doc['id']}). You can view the document here: {get_url_link(updated_doc)}")

Updated document (3559. You can view the document here: https://rspace.ntnu.no/globalId/SD3559/


## Get files from a document and do some changes

First, let us download all the files attatched to the document we just created. If you ran parts of the code earlier several times, you might end up with more than one file.

In [63]:
document = client.get_document(updated_doc['id'])
for field in document['fields']:
    files = field.get('files')
    print(f'Field {field["name"]} refers to {len(files)} file(s):')
    for file in files:
        print(f"\t{file['id']}\t{file['globalId']}\t{file['name']}\t{get_url_link(file)}.")
    print(f"Downloading files...")
    downloaded_files = []
    for file in files:
        path = Path(f"{file['id']}_{file['name']}")
        print(f"\tDownloading {file['name']}({file['id']}) to {path.absolute()}...")
        client.download_file(file['id'], path.absolute())
        print(f"\t\tFinished downloading file.")
        downloaded_files.append(path)

Field Data refers to 1 file(s):
	3562	GL3562	test.png	https://rspace.ntnu.no/globalId/GL3562/.
		Finished downloading file.


Next, let us read the .png images back into Python and do some cosmetic changes that does not make sense at all, but serves to show how you can work with data you have already put on RSpace and upload new versions of the files.

In [83]:
for file in downloaded_files:
    im = plt.imread(file.absolute())
    modified_image = im.copy()
    modified_image[:, :, 3] = 0.5 #Set opacity to 50%
    modified_image[:, :, 2] = 0 #Set blue channel to 0
    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].imshow(im)
    ax[0].set_title('Original')
    ax[1].imshow(modified_image)
    ax[1].set_title('Modified image')
    fig.suptitle("Modified image comparison")
    plt.tight_layout()
    
    modified_file = file.with_stem(f"{file.stem}_modified")
    fig.savefig(modified_file)
    
    file_id = file.stem.split('_')[0] #file ID is stored in file path. There are more elegant ways to do this....
    
    with modified_file.open('rb') as f:
        updated_file = client.update_file(f, file_id)
    
    print(f"Updated file {updated_file['name']} ({updated_file['id']}). No changes are required in the original document!")

3562_test.png


<IPython.core.display.Javascript object>

Updated file 3562_test_modified.png(3562). No changes are required in the original document!


We can also do changes to the text. However, this is not very convenient unless you are working with forms with well-defined fields. For our basic document, for now let us make all the text italic.

In [95]:
content = "<em>" + client.get_document(document['id'])['fields'][0]['content'] + "</em>"
client.update_document(document['id'], fields=[{"content": content}])

{'id': 3559,
 'globalId': 'SD3559',
 'name': 'Test',
 'created': '2025-01-17T15:15:28.452Z',
 'lastModified': '2025-01-18T12:43:46.894Z',
 'parentFolderId': 2303,
 'signed': False,
 'tags': 'API,Test',
 'tagMetaData': 'API,Test',
 'form': {'id': 65536,
  'globalId': 'FM65536',
  'stableId': '1724164328846null',
  'version': 0,
  'name': 'Basic Document',
  'tags': None,
  'formState': 'PUBLISHED',
  'accessControl': {'ownerPermissionType': 'WRITE',
   'groupPermissionType': 'NONE',
   'worldPermissionType': 'READ'},
  'iconId': -1,
  '_links': [{'link': 'https://rspace.ntnu.no/api/v1/forms/65536',
    'rel': 'self'}]},
 'owner': {'id': 196610,
  'username': 'emilc@ntnu.no',
  'email': 'emil.christiansen@ntnu.no',
  'firstName': 'Emil Frang',
  'lastName': 'Christiansen',
  'homeFolderId': 615,
  'workbenchId': None,
  'hasPiRole': True,
  'hasSysAdminRole': False,
  '_links': []},
 'fields': [{'id': 393277,
   'globalId': 'FD393277',
   'name': 'Data',
   'type': 'text',
   'content': 

# Creating documents from an experiment

Up to now, we have only worked with sample files and "dummy" data that does not really reflect how RSpace can be incorporated into your workflows. The biggest benefit of RSpace is that it can serve as a stable space for your results (and in some cases your raw data) _as well as_ for your notes and thoughts about your data. This can help you remember what you did half a year ago, and why you did it a particular way and not another way. Another great benefit is that RSpace enables you to share your work with your colleagues. This means that if you are struggeling with parts of the analysis for instance, you can share your work with someone else who can do that particular analysis and upload the results and explain them directly in the documents you are working on all the time!

In this part, we will base our work on an example experiment that already consists of calibrated .hspy files. We could have started out with raw .mib files, but due to size limitations etc. it is easier to work directly with the .hspy files. While RSpace can handle large files, it is advised to stay below 2GB filesizes.

We will start by looking at the cameralength calibration data (in the form of SAED patterns), and plot and output calibration values to ensure our calibrations are reasonable. We will also evaluate any distortions to our ring patterns. Next, we will measure the precession angle from a stack of diffraction patterns acquired with precession on but descan off. Finally, we will look at a SPED stack and create VBFs and VADFs of particular angles.

## Part 1 - Calibrations

Let us start by creating an RSpace document where we will continously update with our results

In [196]:
document = client.create_document(name="Example diffraction experiment", parent_folder_id=None, tags=["API", "Example", "Diffraction", "SPED", "SAED", "TEM"])

In [98]:
datapath = Path(r'../example_experiment')
text = f"""
<h1>Example diffraction experiment</h1>
<h2>SAED calibration data</h2>
<p> Working on SAED patterns from {datapath.absolute()}</p>
"""
client.append_content(document['id'], text)

In [100]:
saed = hs.load(str(datapath/'SAED_*.hspy'), stack=False)

In [210]:
#Set some metadata
uploaded_files = []
for pattern in saed:
    camera_length = int(pattern.metadata.General.original_filename.split('_')[-1].replace('cm.mib', ''))
    pattern.set_experimental_parameters(
        beam_energy=200,
        camera_length=camera_length
    )
    pattern.metadata.General.title = f"{camera_length} cm"
    
    path = (datapath/pattern.metadata.General.original_filename).with_suffix('.hspy')
    
    pattern.save(path, overwrite=True)
    

    with path.open('rb') as f:
        uploaded_files.append(client.upload_file(f, caption=f"SAED {camera_length} cm: {path.stem}"))

text = f"""
SAED patterns:
<ul>
"""
text += '\n'.join([f"<li>{file['globalId']}</li>" for file in uploaded_files])
text += "\n</ul>"

print(text)

    


SAED patterns:
<ul>
<li>GL3618</li>
<li>GL3619</li>
<li>GL3620</li>
<li>GL3621</li>
<li>GL3622</li>
<li>GL3623</li>
<li>GL3624</li>
<li>GL3625</li>
<li>GL3626</li>
<li>GL3627</li>
<li>GL3628</li>
</ul>


In [208]:
uploaded_files

[{'id': 3606,
  'globalId': 'GL3606',
  'name': 'SAED_NBD_alpha5_spot0p5nm_80cm.hspy',
  'caption': 'SAED 80 cm: SAED_NBD_alpha5_spot0p5nm_80cm',
  'contentType': 'application/octet-stream',
  'created': '2025-01-19T13:00:49.376Z',
  'size': 180102,
  'version': 1,
  '_links': [{'link': 'https://rspace.ntnu.no/api/v1/files/3606',
    'rel': 'self'},
   {'link': 'https://rspace.ntnu.no/api/v1/files/3606/file',
    'rel': 'enclosure'}]}]

In [204]:
text

'\nSAED Patterns:\n<ul>\n<li><fileId=GL3583></li>\n</ul>'

In [197]:
_ = hs.plot.plot_images(saed, axes_decor='off', colorbar=None, norm='symlog', label='auto')
fig = plt.gcf()

fig.savefig(datapath / 'SAED.png')

with (datapath/'SAED.png').open('rb') as f: 
    file = client.upload_file(f, caption="SAED calibration patterns")

<IPython.core.display.Javascript object>

In [ ]:
client.append_content(document['id'], f"SAED patterns: <fileId={file['id']}>")

In [172]:
def cameralength(pattern, pixel_size = 55.0):
    """
    Return the actual cameralength of a diffraction pattern given a physical pixel size
    
    Parameters
    ----------
    pattern : pxm.signals.ElectronDiffraction
        The electron diffraction pattern to calculate the cameralength for.
    pixel_size : float. Optional.
        The physical pixel size of the detector in microns. Default is the pixel size for the Merlin EM 1S (55 um).

    Returns
    -------
    cameralength : float
        The actual cameralength of the diffraction pattern.
    """
    
    beam_energy = pattern.metadata.Acquisition_instrument.TEM.beam_energy
    wavelength = pxm.utils.calibration.get_electron_wavelength(beam_energy)
    return pixel_size * 1E-6 / (wavelength*pattern.axes_manager[-1].scale) *1E2

def cameralength2scale(cameralength, pixel_size = 55.0, beam_energy=200, err=0.0):
    """
    Calculates the corresponding scale given a cameralength, physical pixel size, and the beam energy.
    
    Also estimates the error in the scale given an error in the cameralength by applying Gauss Error Propagation.
    
    Parameters
    ----------
    cameralength : float
        The actual cameralength of a diffraction pattern in cm.
    pixel_size : float. Optional.
        The physical pixel size of the camera in microns. Default is the pixel size for the Merlin EM 1S (55 um).
    beam_energy : float
        The electron beam energy in kV. Default is 200 kV.
    err : float
        The standard error of the cameralength. Default is 0.

    Returns
    -------
    scale : float
        The scale of the diffraction pattern in 1/Å/px
    scale_error : float
        The estimated error of the diffraction pattern scale.
    """
    
    wavelength = pxm.utils.calibration.get_electron_wavelength(beam_energy)
    
    scale = pixel_size*1E-6 / (wavelength * cameralength*1E-2)
    
    scale_error = scale * err / cameralength #Applied Gauss Error Propagation
    
    return scale, scale_error

Calculate the actual cameralengths

In [151]:
actual_cameralengths = np.array([cameralength(pattern) for pattern in saed])
nominal_cameralengths = np.array([pattern.metadata.Acquisition_instrument.TEM.Detector.Diffraction.camera_length for pattern in saed])

Fitted parameters of y=ax+b:
	a=1.7 +/- 3.1
	b=3.1 +/- 0.19
	R^2=0.9999


<IPython.core.display.Javascript object>

Do some curve fitting

In [ ]:
from scipy.optimize import curve_fit

def line(x, a, b):
    return a * x + b

popt, pcov = curve_fit(line, nominal_cameralengths, actual_cameralengths)
perr = np.sqrt(np.diag(pcov))
residuals = actual_cameralengths - line(nominal_cameralengths, *popt)
ss_residuals = np.sum(residuals ** 2)
ss_total = np.sum((actual_cameralengths - np.mean(actual_cameralengths)) ** 2)
r_squared = 1 - (ss_residuals/ss_total)
fit_summary = f"Fitted parameters of y=ax+b:\n\ta={popt[0]:.2g} +/- {popt[1]:.2g}\n\tb={popt[1]:.2g} +/- {perr[1]:.2g}\n\tR^2={r_squared:.4g}"

print(fit_summary)

with (datapath/'summary.txt').open('a+') as f:
    f.write(fit_summary)


Plot the calibrations and the curve fit

In [173]:
fig, ax = plt.subplots()
ax.plot(nominal_cameralengths, line(nominal_cameralengths, *popt), 'r-', label=f'Estimate $({popt[0]:.2g}\pm{perr[0]:.2g})*x + {popt[1]:.2g}\pm{perr[1]:.2g}$')
ax.plot(nominal_cameralengths, actual_cameralengths, 'bx', label='Meaurements')
ax.set_title(f'Calibration plot ($R^2={r_squared:.6g}$)')
ax.set_xlabel('Nominal cameralength (cm)')
ax.set_ylabel('Actual cameralength (cm)')
plt.legend()

fig.savefig(datapath/f'calibration_plot.png')

<IPython.core.display.Javascript object>

Use the estimate to evaulate the accuracy of the diffraction pattern scales

In [192]:
estimated_cameralengths = line(nominal_cameralengths, *popt)
estimated_cameralengths_err = estimated_cameralengths * popt[0]*perr[0] #Error propagation on linear relationship

scales = [pattern.axes_manager[-1].scale for pattern in saed]
calibration_table = [(nom_cl, meas_cl, cl, err, s) + cameralength2scale(cl, err=err) for nom_cl, meas_cl, cl, err, s in zip(nominal_cameralengths, actual_cameralengths, estimated_cameralengths, estimated_cameralengths_err, scales)]

table = tabulate(calibration_table, headers=['Nominal CL (cm)', 'Measured CL (cm)', 'CL (cm)', 'Estimated CL error (cm)', 'Measured scale (1/Å)', 'Scale (1/Å)', 'Error (1/Å)'])

print(f"Results of calibration estimates:\n{table}")

Results of calibration estimates:
  Nominal CL (cm)    Measured CL (cm)    CL (cm)    Estimated CL error (cm)    Measured scale (1/Å)    Scale (1/Å)    Error (1/Å)
-----------------  ------------------  ---------  -------------------------  ----------------------  -------------  -------------
                8             17.1424    16.849                    0.139263              0.0127931      0.0130158     0.00010758
               10             20.6117    20.2899                   0.167702              0.0106398      0.0108085     8.93361e-05
               12             23.8769    23.7307                   0.196142              0.00918478     0.00924137    7.63829e-05
               15             28.9788    28.8919                   0.238801              0.00756774     0.0075905     6.27379e-05
               20             36.954     37.4939                   0.309899              0.00593451     0.00584905    4.83443e-05
               25             45.6028    46.096          

In [193]:
dataframe = pd.DataFrame(calibration_table, columns=['Nominal CL (cm)', 'Measured CL (cm)', 'Estimated CL (cm)', 'Estimated CL error (cm)', 'Measured scale (1/Å)', 'Estimated scale (1/Å)', 'Estimated scale error (1/Å)'])

In [194]:
dataframe

,Nominal CL (cm),Measured CL (cm),Estimated CL (cm),Estimated CL error (cm),Measured scale (1/Å),Estimated scale (1/Å),Estimated scale error (1/Å)
0,8,17.142382,16.849048,0.139263,0.012793,0.013016,0.000108
1,10,20.611673,20.289865,0.167702,0.010640,0.010809,0.000089
2,12,23.876889,23.730681,0.196142,0.009185,0.009241,0.000076
3,15,28.978788,28.891906,0.238801,0.007568,0.007590,0.000063
4,20,36.954008,37.493948,0.309899,0.005935,0.005849,0.000048
5,25,45.602819,46.095990,0.380998,0.004809,0.004758,0.000039
6,30,54.696263,54.698032,0.452097,0.004009,0.004009,0.000033
7,40,71.901191,71.902115,0.594294,0.003050,0.003050,0.000025
8,50,88.776972,89.106199,0.736491,0.002470,0.002461,0.000020
9,60,106.824627,106.310283,0.878688,0.002053,0.002063,0.000017


In [195]:
dataframe.to_csv(datapath / 'calibration_summary.csv', index=False)